In [ ]:
!nvidia-smi

Mon Sep  7 00:09:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import math

%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.image import per_image_standardization, resize
from tensorflow import keras
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers

import string

path = '/content/gdrive/My Drive/Dacon/ComputerVision/'
path_train = path + 'train.csv'
path_test = path + 'test.csv'
path_submission = path + 'submission.csv'

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
seed = 0
tf.random.set_seed(seed)
np.random.seed(seed)

## Global Variables ##
ImageGen_coeff = 10
N_dim = 224
epochs_num = 25 # tried 50, too. both failed to get high score.
verbose = 2
batch_size = 32

def get_progressbar_str(progress):
    MAX_LEN = 30
    BAR_LEN = int(MAX_LEN * progress)
    return ('Progress:[' + '=' * BAR_LEN +
            ('>' if BAR_LEN < MAX_LEN else '') +
            ' ' * (MAX_LEN - BAR_LEN) +
            '] %.1f%%' % (progress * 100.))

Data Pipelines
-----------

In [ ]:
def prepare_datasets():
    '''
    ImageGen_coeff (Global Variable) : number of ImageDataGenerator generation per image
    N_dim (Global Variable) : we'll change the size of the image from (28,28) to (N_dim,N_dim)

    Since we don't have enough RAM for the resized image of test_pixel, we just return non-resized, non-standardized image.
    When we generate each batchs by making use of the tf.utils.Sequential class, we'll resize and standardize test_pixel.
    '''
    train_raw = pd.read_csv(path_train)
    test_raw = pd.read_csv(path_test)

    letter_hash = dict(zip(string.ascii_uppercase, [[1 if i == j else 0 for j in range(26)] for i in range(26)]))
    letter_hash_func = lambda letter : letter_hash[letter]

    pix = train_raw.iloc[:,3:].values.astype('int32').reshape(-1,28,28,1)
    fix = train_raw.iloc[:,1:3].values
    datagenerator = ImageDataGenerator(rotation_range=10, zoom_range=0.10, width_shift_range=0.1,height_shift_range=0.1)
    gen = datagenerator.flow(pix, fix, shuffle=False, batch_size=32)
    pixel, fixed, batch_index, limit = [], [], 0, 64 * ImageGen_coeff
    while batch_index <= limit:
        try:
            data = gen.next()
            pixel += list(data[0])
            fixed += list(data[1])
            batch_index += 1
        except:
            print("ImageGeneratorError")
            break
    # print(np.asarray(pixel).shape)
    with tf.device('/device:GPU:0'):
        train_pixel = resize(np.asarray(pixel), [N_dim,N_dim], method = tf.image.ResizeMethod.BICUBIC)
        train_pixel = per_image_standardization(train_pixel).numpy()
        train_label = np.asarray(list(map(letter_hash_func, np.asarray(fixed)[:,1])))
        train_answer = to_categorical(np.asarray(fixed)[:,0], 10)

    test_pixel = test_raw.iloc[:,2:].values.astype('int32').reshape(-1,28,28,1)
    test_label = np.asarray(list(map(letter_hash_func, test_raw.iloc[:,1].values)))
    return train_pixel, train_label, train_answer, test_pixel, test_label

train_pixel, train_label, train_answer, test_pixel, test_label = prepare_datasets()
print(train_pixel.shape, train_label.shape, train_answer.shape, test_pixel.shape, test_label.shape)

(20512, 224, 224, 1) (20512, 26) (20512, 10) (20480, 28, 28, 1) (20480, 26)


In [ ]:
train_label = train_label.astype('float32')
test_label = test_label.astype('float32')

teacher 1
-------

In [ ]:
'''
Since I'm using VRAM 16GB, it is not sufficient for generating EfficientNetB6 or higher. We use from B2 to B5.
'''

# teacher 1

def teacher1(pixel, label, answer):
    ModelName = 'teacher_1_EfficientNetB2_epoch25'
    with tf.device('/device:GPU:0'):
        model = tf.keras.applications.EfficientNetB2(include_top=False, weights=None, input_tensor=None, input_shape=(N_dim,N_dim,1),pooling=None)
        cnn_mid = layers.GlobalAveragePooling2D()(model.output)
        cnn_out = layers.Dense(128, activation = 'relu')(cnn_mid)

        dense_input = layers.Input(shape=(26,))
        dense_mid = layers.Dense(52, activation = 'relu')(dense_input)
        dense_output = layers.Dense(52, activation = 'relu')(dense_mid)
        dense_model = tf.keras.Model(inputs=dense_input, outputs=dense_output)

        concatenated = layers.concatenate([cnn_out, dense_model.output])
        concatenated = layers.Dense(32, activation='relu')(concatenated)
        concat_output = layers.Dense(10, activation='softmax')(concatenated)
        concat_model = tf.keras.models.Model([model.input, dense_input], concat_output)
        concat_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        #print(concat_model.summary())
        history = concat_model.fit([pixel, label], answer, epochs=epochs_num, verbose=verbose)
    concat_model.save(path + ModelName + '.h5')
    print(
        f"CNN: Epochs={epochs_num:d}, " +
        f"Train accuracy={max(history.history['accuracy']):.5f}, "
    )
    return concat_model

teacher = teacher1(train_pixel, train_label, train_answer)

Epoch 1/25
641/641 - 138s - loss: 1.8221 - accuracy: 0.3379
Epoch 2/25
641/641 - 138s - loss: 0.8306 - accuracy: 0.7235
Epoch 3/25
641/641 - 138s - loss: 0.4830 - accuracy: 0.8400
Epoch 4/25
641/641 - 138s - loss: 0.2750 - accuracy: 0.9080
Epoch 5/25
641/641 - 138s - loss: 0.1946 - accuracy: 0.9350
Epoch 6/25
641/641 - 138s - loss: 0.1507 - accuracy: 0.9496
Epoch 7/25
641/641 - 138s - loss: 0.1252 - accuracy: 0.9590
Epoch 8/25
641/641 - 138s - loss: 0.0960 - accuracy: 0.9681
Epoch 9/25
641/641 - 138s - loss: 0.0832 - accuracy: 0.9728
Epoch 10/25
641/641 - 138s - loss: 0.0696 - accuracy: 0.9771
Epoch 11/25
641/641 - 138s - loss: 0.0704 - accuracy: 0.9784
Epoch 12/25
641/641 - 138s - loss: 0.0645 - accuracy: 0.9793
Epoch 13/25
641/641 - 138s - loss: 0.0588 - accuracy: 0.9811
Epoch 14/25
641/641 - 139s - loss: 0.0528 - accuracy: 0.9824
Epoch 15/25
641/641 - 138s - loss: 0.0451 - accuracy: 0.9859
Epoch 16/25
641/641 - 137s - loss: 0.0511 - accuracy: 0.9838
Epoch 17/25
641/641 - 139s - loss

If disconnected...
-----

In [ ]:
# teacher = tf.keras.models.load_model(path+'teacher_1_EfficientNetB2_epoch25.h5')
# teacher = tf.keras.models.load_model(path+'teacher_2_EfficientNetB3_epoch25.h5')
teacher = tf.keras.models.load_model(path+'teacher_3_EfficientNetB4_epoch25.h5')

In [ ]:
class Handouts_for_Lecture(Sequence):
    def __init__(self,train_pixel, train_label, train_answer, test_pixel, test_label, batch_size, teacher):
        self.train_pixel = train_pixel
        self.train_label = train_label
        self.train_answer = train_answer
        self.test_pixel = test_pixel
        self.test_label = test_label

        self.total_data = len(train_label) + len(test_label)
        self.batch_size = batch_size
        self.length = math.ceil(self.total_data / batch_size)
        self.batch_size_train = math.ceil(len(train_label) / self.length)
        self.batch_size_test = batch_size - self.batch_size_train
        
        ratio = len(test_label) // len(train_label)

        idxs_train = np.arange(len(train_label))
        idxs_test = np.arange(len(test_label))
        np.random.shuffle(idxs_train)
        np.random.shuffle(idxs_test)
        self.idxs_train = idxs_train
        self.idxs_test = idxs_test

        self.teacher = teacher

    def __getitem__(self,idx):
        trainings = self.get_test(idx) + self.get_train(idx)
        # for shuffling
        temp = np.arange(len(trainings))
        np.random.shuffle(temp)
        trainings = np.asarray([trainings[i] for i in temp])

        pixel = np.asarray([elt[0] for elt in trainings])
        label = np.asarray([elt[1] for elt in trainings])
        answer = np.asarray([elt[2] for elt in trainings])

        return [pixel,label],answer

    def __len__(self):
        return self.length

    def get_train(self,idx):
        i = idx * self.batch_size_train
        if i + self.batch_size_train > len(self.idxs_train):
            index_window = [j for j in range(i,len(self.idxs_train))]
        else:
            index_window = [j for j in range(i, i + self.batch_size_train)]
        pixel = np.asarray([self.train_pixel[self.idxs_train[k]] for k in index_window])
        label = np.asarray([self.train_label[self.idxs_train[k]] for k in index_window])
        answer = np.asarray([self.train_answer[self.idxs_train[k]] for k in index_window])

        return list(zip(pixel,label,answer))

    def get_test(self,idx):
        i = idx * self.batch_size_test
        if i + self.batch_size_test > len(self.idxs_test):
            index_window = [j for j in range(i,len(self.idxs_test))]
        else:
            index_window = [j for j in range(i, i + self.batch_size_test)]
        pixel = np.asarray([self.test_pixel[self.idxs_test[k]] for k in index_window])
        label = np.asarray([self.test_label[self.idxs_test[k]] for k in index_window])

        with tf.device('/device:GPU:0'):
            pixel = resize(pixel, [N_dim,N_dim], method = tf.image.ResizeMethod.BICUBIC)
            pixel = per_image_standardization(pixel).numpy()
            predicted = self.teacher.predict([pixel,label])
        answer = np.asarray([int(np.argmax(x)) for x in predicted])
        answer = to_categorical(answer, 10)

        return list(zip(pixel,label,answer))

DataSet = Handouts_for_Lecture(train_pixel, train_label, train_answer, test_pixel, test_label, batch_size, teacher)

teacher 2
--------

In [ ]:
# teacher 2

def teacher2(DataSet):
    ModelName = 'teacher_2_EfficientNetB3_epoch25'
    with tf.device('/device:GPU:0'):
        model = tf.keras.applications.EfficientNetB3(include_top=False, weights=None, input_tensor=None, input_shape=(N_dim,N_dim,1),pooling=None)
        cnn_mid = layers.GlobalAveragePooling2D()(model.output)
        cnn_out = layers.Dense(128, activation = 'relu')(cnn_mid)

        dense_input = layers.Input(shape=(26,))
        dense_mid = layers.Dense(52, activation = 'relu')(dense_input)
        dense_output = layers.Dense(52, activation = 'relu')(dense_mid)
        dense_model = tf.keras.Model(inputs=dense_input, outputs=dense_output)

        concatenated = layers.concatenate([cnn_out, dense_model.output])
        concatenated = layers.Dense(32, activation='relu')(concatenated)
        concat_output = layers.Dense(10, activation='softmax')(concatenated)
        concat_model = tf.keras.models.Model([model.input, dense_input], concat_output)
        concat_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        #print(concat_model.summary())

        history = concat_model.fit(DataSet, batch_size = 1, epochs=epochs_num, verbose=verbose)
    concat_model.save(path + ModelName + '.h5')
    print(
        f"CNN: Epochs={epochs_num:d}, " +
        f"Train accuracy={max(history.history['accuracy']):.5f}, "
    )
    return concat_model

teacher = teacher2(DataSet)
DataSet = Handouts_for_Lecture(train_pixel, train_label, train_answer, test_pixel, test_label, batch_size, teacher)

Epoch 1/25
1281/1281 - 353s - loss: 1.2952 - accuracy: 0.5510
Epoch 2/25
1281/1281 - 354s - loss: 0.5123 - accuracy: 0.8392
Epoch 3/25
1281/1281 - 357s - loss: 0.3755 - accuracy: 0.8806
Epoch 4/25
1281/1281 - 354s - loss: 0.3124 - accuracy: 0.9000
Epoch 5/25
1281/1281 - 352s - loss: 0.2652 - accuracy: 0.9152
Epoch 6/25
1281/1281 - 349s - loss: 0.2280 - accuracy: 0.9257
Epoch 7/25
1281/1281 - 354s - loss: 0.1969 - accuracy: 0.9376
Epoch 8/25
1281/1281 - 355s - loss: 0.1678 - accuracy: 0.9449
Epoch 9/25
1281/1281 - 351s - loss: 0.1445 - accuracy: 0.9530
Epoch 10/25
1281/1281 - 351s - loss: 0.1259 - accuracy: 0.9587
Epoch 11/25
1281/1281 - 354s - loss: 0.1188 - accuracy: 0.9599
Epoch 12/25
1281/1281 - 353s - loss: 0.0995 - accuracy: 0.9665
Epoch 13/25
1281/1281 - 354s - loss: 0.0935 - accuracy: 0.9689
Epoch 14/25
1281/1281 - 352s - loss: 0.0800 - accuracy: 0.9739
Epoch 15/25
1281/1281 - 352s - loss: 0.0749 - accuracy: 0.9756
Epoch 16/25
1281/1281 - 349s - loss: 0.0709 - accuracy: 0.9771
E

In [ ]:
# teacher 3

def teacher3(DataSet):
    ModelName = 'teacher_3_EfficientNetB4_epoch25'
    with tf.device('/device:GPU:0'):
        model = tf.keras.applications.EfficientNetB4(include_top=False, weights=None, input_tensor=None, input_shape=(N_dim,N_dim,1),pooling=None)
        cnn_mid = layers.GlobalAveragePooling2D()(model.output)
        cnn_out = layers.Dense(128, activation = 'relu')(cnn_mid)

        dense_input = layers.Input(shape=(26,))
        dense_mid = layers.Dense(52, activation = 'relu')(dense_input)
        dense_output = layers.Dense(52, activation = 'relu')(dense_mid)
        dense_model = tf.keras.Model(inputs=dense_input, outputs=dense_output)

        concatenated = layers.concatenate([cnn_out, dense_model.output])
        concatenated = layers.Dense(32, activation='relu')(concatenated)
        concat_output = layers.Dense(10, activation='softmax')(concatenated)
        concat_model = tf.keras.models.Model([model.input, dense_input], concat_output)
        concat_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        #print(concat_model.summary())
        history = concat_model.fit(DataSet, batch_size = 1, epochs=epochs_num, verbose=verbose)
    concat_model.save(path + ModelName + '.h5')
    print(
        f"CNN: Epochs={epochs_num:d}, " +
        f"Train accuracy={max(history.history['accuracy']):.5f}, "
    )
    return concat_model
    
teacher = teacher3(DataSet)
DataSet = Handouts_for_Lecture(train_pixel, train_label, train_answer, test_pixel, test_label, batch_size, teacher)

Epoch 1/25
1281/1281 - 433s - loss: 1.2522 - accuracy: 0.5638
Epoch 2/25
1281/1281 - 432s - loss: 0.5057 - accuracy: 0.8442
Epoch 3/25
1281/1281 - 432s - loss: 0.3836 - accuracy: 0.8797
Epoch 4/25
1281/1281 - 433s - loss: 0.3125 - accuracy: 0.9007
Epoch 5/25
1281/1281 - 427s - loss: 0.2606 - accuracy: 0.9157
Epoch 6/25
1281/1281 - 432s - loss: 0.2279 - accuracy: 0.9274
Epoch 7/25
1281/1281 - 433s - loss: 0.1963 - accuracy: 0.9368
Epoch 8/25
1281/1281 - 432s - loss: 0.1667 - accuracy: 0.9456
Epoch 9/25
1281/1281 - 433s - loss: 0.1493 - accuracy: 0.9511
Epoch 10/25
1281/1281 - 431s - loss: 0.1251 - accuracy: 0.9583
Epoch 11/25
1281/1281 - 432s - loss: 0.1103 - accuracy: 0.9645
Epoch 12/25
1281/1281 - 432s - loss: 0.1011 - accuracy: 0.9663
Epoch 13/25
1281/1281 - 432s - loss: 0.0844 - accuracy: 0.9718
Epoch 14/25
1281/1281 - 431s - loss: 0.0798 - accuracy: 0.9730
Epoch 15/25
1281/1281 - 433s - loss: 0.0754 - accuracy: 0.9752
Epoch 16/25
1281/1281 - 428s - loss: 0.0645 - accuracy: 0.9788
E

In [ ]:
# teacher 4

def teacher4(DataSet):
    ModelName = 'teacher_4_EfficientNetB5_epoch25'
    with tf.device('/device:GPU:0'):
        model = tf.keras.applications.EfficientNetB5(include_top=False, weights=None, input_tensor=None, input_shape=(N_dim,N_dim,1),pooling=None)
        cnn_mid = layers.GlobalAveragePooling2D()(model.output)
        cnn_out = layers.Dense(128, activation = 'relu')(cnn_mid)

        dense_input = layers.Input(shape=(26,))
        dense_mid = layers.Dense(52, activation = 'relu')(dense_input)
        dense_output = layers.Dense(52, activation = 'relu')(dense_mid)
        dense_model = tf.keras.Model(inputs=dense_input, outputs=dense_output)

        concatenated = layers.concatenate([cnn_out, dense_model.output])
        concatenated = layers.Dense(32, activation='relu')(concatenated)
        concat_output = layers.Dense(10, activation='softmax')(concatenated)
        concat_model = tf.keras.models.Model([model.input, dense_input], concat_output)
        concat_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        #print(concat_model.summary())
        history = concat_model.fit(DataSet, batch_size = 1, epochs=epochs_num, verbose=verbose)
    concat_model.save(path + ModelName + '.h5')
    print(
        f"CNN: Epochs={epochs_num:d}, " +
        f"Train accuracy={max(history.history['accuracy']):.5f}, "
    )
    return concat_model
    
teacher = teacher4(DataSet)

Epoch 1/25
1281/1281 - 586s - loss: 1.5195 - accuracy: 0.4558
Epoch 2/25
1281/1281 - 577s - loss: 0.6765 - accuracy: 0.7831
Epoch 3/25
1281/1281 - 586s - loss: 0.6974 - accuracy: 0.7762
Epoch 4/25
1281/1281 - 587s - loss: 0.4371 - accuracy: 0.8641
Epoch 5/25
1281/1281 - 592s - loss: 0.3456 - accuracy: 0.8910
Epoch 6/25
1281/1281 - 594s - loss: 0.2939 - accuracy: 0.9071
Epoch 7/25
1281/1281 - 588s - loss: 0.2669 - accuracy: 0.9152
Epoch 8/25
1281/1281 - 596s - loss: 0.2336 - accuracy: 0.9242
Epoch 9/25
1281/1281 - 590s - loss: 0.2033 - accuracy: 0.9349
Epoch 10/25
1281/1281 - 589s - loss: 0.1794 - accuracy: 0.9419
Epoch 11/25
1281/1281 - 586s - loss: 0.1594 - accuracy: 0.9494
Epoch 12/25
1281/1281 - 594s - loss: 0.1402 - accuracy: 0.9543
Epoch 13/25
1281/1281 - 598s - loss: 0.1218 - accuracy: 0.9590
Epoch 14/25
1281/1281 - 577s - loss: 0.1094 - accuracy: 0.9635
Epoch 15/25
1281/1281 - 595s - loss: 0.0970 - accuracy: 0.9672
Epoch 16/25
1281/1281 - 589s - loss: 0.0858 - accuracy: 0.9714
E

In [ ]:
class ComputerVision_Data_test():
    def __init__(self, pixel, label, batch_size):
        self.pixel = pixel
        self.label =  label
        self.batch_size = batch_size
        self.length = math.ceil(len(label) // batch_size)

    def __getitem__(self, idx):
        i = idx * self.batch_size
        if i + self.batch_size > len(self.label):
            pixel = self.pixel[i:]
            label = self.label[i:]
        else:
            pixel = self.pixel[i:i+self.batch_size]
            label = self.label[i:i+self.batch_size]
        with tf.device('/device:GPU:0'):
            pixel = resize(pixel, [224,224], method=tf.image.ResizeMethod.BICUBIC)
            pixel = per_image_standardization(pixel).numpy()

        return [pixel, label]

    def __len__(self):
        return self.length


test_dataset = ComputerVision_Data_test(test_pixel, test_label, batch_size = 1000)

In [ ]:
def split_testing(model,ModelName, test_dataset, batch_idxs):
    batch_size = test_dataset.batch_size
    submission = pd.read_csv(path_submission)
    inc = 0
    L = len(batch_idxs)
    for batch_idx in batch_idxs:
        target = test_dataset[batch_idx]
        real_idxs = [i for i in range(batch_size * batch_idx, batch_size * batch_idx + len(target[0])) ]
        with tf.device('/device:GPU:0'):
            predicted = model.predict(target)
        res = [int(np.argmax(x)) for x in predicted]
        for idx_temp, real in enumerate(real_idxs):
            submission.iloc[real,1] = res[idx_temp]
        inc += 1
        sys.stderr.write('\r\033[K' + get_progressbar_str(inc/L))
        sys.stderr.flush()
    sys.stderr.write('\n')
    sys.stderr.flush()
    
    submission.to_csv(path+'submission_'+ModelName+'_epoch25.csv',index = False) # FOR EPOCH 25

    return submission

batch_idxs = [i for i in range(0, len(test_dataset))]
for ModelName in ['teacher_1_EfficientNetB2_epoch25','teacher_2_EfficientNetB3_epoch25','teacher_3_EfficientNetB4_epoch25','teacher_4_EfficientNetB5_epoch25']:
    print(ModelName)
    teacher = tf.keras.models.load_model(path+ModelName + '.h5')
    submission = split_testing(teacher,ModelName, test_dataset, batch_idxs)

teacher_1_EfficientNetB2_epoch25


Progress:[==============================] 100.0%


teacher_2_EfficientNetB3_epoch25


Progress:[==============================] 100.0%


teacher_3_EfficientNetB4_epoch25


Progress:[==============================] 100.0%


teacher_4_EfficientNetB5_epoch25


Progress:[==============================] 100.0%
